In [ ]:
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
import os
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv

c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

#### Defining the model

In [3]:
llm = ChatGroq(
            temperature=1, 
            groq_api_key = os.environ["GROQ_API_KEY"], 
            model_name="llama-3.1-8b-instant",
            max_tokens=600,
            top_p=0.90,
        #     frequency_penalty=1,
        #     presence_penalty=1,
    )

c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_groq\chat_models.py:364: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  warnings.warn(


#### Embbeding and using vector database

In [4]:
embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct",model_kwargs={"device": "cuda"})
chroma_db = Chroma(
    persist_directory="./chroma_capstone_db_new",
    embedding_function=embedding,
    collection_name="Tomato"  # Specify which collection to load
)

### Chromadb as Retriever using mmr

In [5]:
question = "give me the cure for tomato plant in Kashmir in the summer months"
chroma_retriever = chroma_db.as_retriever(search_type="mmr", search_kwargs={"k": 6, "fetch_k":12})

chroma_retriever.get_relevant_documents(question)

C:\Users\HP\AppData\Local\Temp\ipykernel_24156\3181224039.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chroma_retriever.get_relevant_documents(question)


[Document(id='3eb7f99e-c0ae-41a9-9b49-33183493dabb', metadata={'category': 'Tomato'}, page_content='DistrictName:JAMMU\nStateName:JAMMU AND KASHMIR\nSeason_English:Summer Cropping Season\nMonth:July\nDisease:Late_blight\nQueryText:inforation regarding control of late blight in tomato\nKccAns:spray indofil m45  25mllitre of water'),
 Document(id='015d7c8f-04c0-46c9-b5a4-2c5183412531', metadata={'category': 'Tomato'}, page_content='DistrictName:JAMMU\nStateName:JAMMU AND KASHMIR\nSeason_English:Summer Cropping Season\nMonth:April\nDisease:Late_blight\nQueryText:farmer seeks information about the control of  late blight in tomato\nKccAns:spray mancozeb 75 wp  25 gm  litre of water'),
 Document(id='294f8de4-bb06-4dc3-95f4-a176f171ee5c', metadata={'category': 'Tomato'}, page_content='DistrictName:BADGAM\nStateName:JAMMU AND KASHMIR\nSeason_English:nan\nMonth:June\nDisease:Late_blight\nQueryText:late blight in tomato\nKccAns:mancozeb 2gltr of water'),
 Document(id='54169f6e-84c5-4e53-9499-3d

#### Prompting

In [6]:
prompt_template = """
You are an agricultural assistant specialized in answering questions about plant diseases.  
Your task is to provide answers strictly based on the provided context when possible.  

Each document contains the following fields:  
- DistrictName  
- StateName  
- Season_English  
- Month  
- Disease  
- QueryText  
- KccAns (this is the official response section from source documents)

Guidelines for answering:
1. If a relevant answer is available in KccAns, use that with minimal changes.
2. Use DistrictName, StateName, Season_English, Month, and Disease only to help interpret the question and select the correct KccAns, but **do not include these details in the final answer unless the question explicitly asks for them**.  
3. If the answer is not available in the context, then rely on your own agricultural knowledge to provide the best possible answer.  
4. Do not invent or assume information when KccAns is present; only fall back to your own knowledge when the context has no suitable answer.  

CONTEXT:
{context}

QUESTION:
{question}

OUTPUT:
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

#### Retriever

In [7]:
h_retrieval_QA1 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chroma_retriever,
    input_key="query",
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [8]:
question = "What are some of the common prescriptions for tomato plants in Karnataka"
print(h_retrieval_QA1.invoke({"query": question})["result"])

Based on the context provided, one document mentions the StateName as KARNATAKA and the Disease as Powdery_mildew, in the Month of May. 

The relevant KccAns is: 
"Kolar, Karnataka, May, Powdery_mildew, Kolar, Karnataka, May, Powdery_mildew, QueryText: tomato powdery mildew, KccAns: spray wettable sulphur 2gliter general tonic 3mlliter"

Considering this, a common prescription for tomato plants in Karnataka (Karnataka) is: 
Spray wettable sulphur 2 grams per liter with a general tonic 3ml per liter.


In [9]:
Q2 = "I have a Fruit borer in my Tomato, what prescriptions would you recommend in Madhya Pradesh"
print(h_retrieval_QA1.invoke({"query": Q2})["result"])

Based on the provided context, I see that there is a relevant answer for Fruit_borer in Tomato in Madhya Pradesh (SHIVPURI, MADHYA PRADESH, October). However, no suitable answer is available in the KccAns section.

In that case, I'll rely on my own agricultural knowledge to provide a possible solution. Unfortunately, none of the given answers can be directly applied as the state is not mentioned in those. But generally, in the case of Fruit borer in Tomatoes, neem oil along with other systemic insecticides such as indoxacarb or emamectin benzoate can be effective in controlling the pest.

However, since we cannot assume the exact pesticide or quantity needed, it's best to consult with a local agricultural expert in Madhya Pradesh who can provide more accurate and up-to-date information about the best treatment for Fruit borer in Tomatoes in the region.

In the absence of specific recommendations for Madhya Pradesh, a general solution for Fruit borer in Tomato could be:
- Indoxacarb at 